## Instalando lib

## Carregamento de Arquivo no Colab

*Observação*:
Normalmente, para facilitar o acesso a arquivos diretamente do Google Drive, utilizaríamos a ferramenta **gdown**, que permite baixar arquivos de maneira rápida e prática. No entanto, devido a restrições de acesso no ambiente do Google Colab e problemas relacionados ao download de arquivos via gdown, esta abordagem não pôde ser utilizada.

Por esse motivo, optamos por uma solução mais simples e direta, onde o usuário pode carregar o arquivo manualmente com apenas um clique. Isso garantirá que o processo funcione corretamente, sem depender de permissões adicionais ou complicações técnicas.
<br><br>
*Instruções Simples*:
<br>**Passo 1**: baixe o arquivo no seguinte link: [Churn_Modelling](https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling?select=Churn_Modelling.csv)
<br>**Passo 2**:Clique no botão abaixo para fazer o upload do arquivo diretamente do seu computador para o Google Colab.
<br>**Passo 3**: Clique no botão Escolher arquivos quando solicitado.
<br>**Passo 4**: Selecione o arquivo Churn_Modelling.csv do seu computador.
O arquivo será carregado automaticamente.

In [20]:
from google.colab import files
import pandas as pd
import io
import tensorflow as tf


# Fazer o upload do arquivo
uploaded = files.upload()

# Carregar o arquivo diretamente a partir do upload
file_name = next(iter(uploaded))  # Pega o nome do arquivo
df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))  # Decodifica o arquivo e carrega no pandas

# Verificar as primeiras linhas do dataset
df.head()


Saving Churn_Modelling.csv to Churn_Modelling (2).csv


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Escolha do Dataset Adequado
Decidimos utilizar o Churn_Modelling.csv, um dataset que não é considerado "toy", como o Iris ou Pima Indians Diabetes. Ele oferece um bom nível de complexidade e volume, com 10.000 amostras e 14 features, e é utilizado para uma tarefa de classificação binária.

In [21]:

# Explorar o dataset
num_samples = df.shape[0]
num_features = df.shape[1]
print(f"Número de amostras: {num_samples}")
print(f"Número de features: {num_features}")
print(df.head())

Número de amostras: 10000
Número de features: 14
   RowNumber  CustomerId   Surname  CreditScore Geography  Gender  Age  \
0          1    15634602  Hargrave          619    France  Female   42   
1          2    15647311      Hill          608     Spain  Female   41   
2          3    15619304      Onio          502    France  Female   42   
3          4    15701354      Boni          699    France  Female   39   
4          5    15737888  Mitchell          850     Spain  Female   43   

   Tenure    Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0       2       0.00              1          1               1   
1       1   83807.86              1          0               1   
2       8  159660.80              3          1               0   
3       1       0.00              2          0               0   
4       2  125510.82              1          1               1   

   EstimatedSalary  Exited  
0        101348.88       1  
1        112542.58       0  
2        113931.57    

## Exploração do Dataset
O dataset contém 10.000 amostras e 14 colunas. A tarefa é prever se um cliente sairá do banco (Exited), que é uma variável binária. As features incluem variáveis categóricas e numéricas, como CreditScore, Geography, Age, entre outras. Algumas dessas variáveis precisam ser codificadas ou normalizadas para serem usadas no modelo de rede neural.

In [22]:
# Verificar a estrutura das colunas e tipos de dados
print(df.info())

# Verificar valores nulos
print(df.isnull().sum())

# Descrição estatística das colunas numéricas
print(df.describe())

# Verificar a distribuição da variável alvo 'Exited'
print(df['Exited'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB
None
RowNumber          0
CustomerId         0
Surname            0
CreditScore        0
Geogr

## Desenvolvimento do Modelo em Keras
Desenvolvemos um modelo sequencial simples com uma única camada Dense, utilizando a função de ativação sigmoid. Este modelo é adequado para classificação binária.

- Sigmoid: Converte a saída em uma probabilidade (entre 0 e 1).
- Binary Crossentropy: Função de perda apropriada para problemas de classificação binária.
- Adam: Otimizador eficiente que ajusta os pesos do modelo durante o treinamento.
- Accuracy e F1-Score: Métricas que avaliam o desempenho do model

In [23]:
# Criar o modelo sequencial
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X_train_scaled.shape[1],))
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

# Exibir o resumo do modelo
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_1 (Dense)                      │ (None, 1)                   │              12 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 12 (48.00 B)

 Trainable params: 12 (48.00 B)

 Non-trainable params: 0 (0.00 B)

**Layer (type)**: A primeira coluna mostra as camadas do modelo. Nesse caso, o modelo contém apenas uma camada, que é uma camada densa (Dense). Camadas densas são aquelas onde cada neurônio está conectado a todos os neurônios da camada anterior.

**Output Shape**: A segunda coluna mostra a forma de saída da camada. Neste caso, a saída é (None, 1). O None indica que o tamanho do lote (batch size) pode ser flexível, ou seja, pode variar dependendo do número de amostras de entrada. O 1 significa que a camada está produzindo uma única saída por amostra (uma vez que é uma tarefa de classificação binária, a saída é uma única probabilidade).

**Param #**: A terceira coluna indica o número total de parâmetros treináveis na camada. Neste caso, a camada tem 12 parâmetros treináveis. Como estamos lidando com uma camada densa, o número de parâmetros é determinado pela fórmula:

### Cálculo dos Parâmetros do Modelo

Para uma camada densa (`Dense`), o número total de parâmetros treináveis é calculado pela fórmula:

**Parâmetros = (número de entradas + 1) × número de neurônios**

- **Número de entradas**: O número de features de entrada do modelo (no nosso caso, 11 após o pré-processamento).
- **1**: Adicionamos um parâmetro extra para o viés (bias).
- **Número de neurônios**: O número de unidades na camada densa (no nosso caso, 1 neurônio para a saída binária).

Assim, no nosso modelo:

**Parâmetros = (11 + 1) × 1 = 12**

No caso, como há 11 entradas (depois de codificar e normalizar os dados) e um único neurônio de saída, os 12 parâmetros incluem 11 pesos e 1 viés (bias).

**Total params**: Esta linha mostra o número total de parâmetros treináveis no modelo, que, neste caso, é 12.

**Trainable params**: Mostra o número de parâmetros que serão ajustados durante o treinamento. Aqui, todos os 12 parâmetros são treináveis.

**Non-trainable params**: Mostra o número de parâmetros que não serão ajustados durante o treinamento. No caso, não há parâmetros não treináveis.

In [26]:
# Treinar o modelo
history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=10, validation_data=(X_test_scaled, y_test), verbose=0)

# Fazer previsões no conjunto de teste
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)

# Calcular a acurácia e o F1-score
from sklearn.metrics import f1_score

accuracy = tf.keras.metrics.BinaryAccuracy()
accuracy.update_state(y_test, y_pred)

f1 = f1_score(y_test, y_pred)

print(f"Acurácia: {accuracy.result().numpy()}")
print(f"F1 Score: {f1}")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Acurácia: 0.8115000128746033
F1 Score: 0.2900188323917137


## Treinamento e Avaliação do Modelo

O modelo foi treinado por 50 épocas com um batch size de 10. Após o treinamento, fizemos previsões no conjunto de teste. Avaliamos o desempenho do modelo utilizando as métricas de acurácia e F1-score.

### Resultados:

- **Acurácia**: 81.1%
- **F1-Score**: 0.29

### Interpretação:

- O modelo apresentou uma acurácia alta (81.1%), o que sugere que ele está fazendo previsões corretas na maioria dos casos. No entanto, o F1-score relativamente baixo (0.29) indica que o modelo pode estar tendo dificuldades em prever corretamente a classe minoritária (clientes que saíram do banco).

### Possíveis Melhorias:

- **Balanceamento de Classes**: Implementar técnicas como oversampling ou undersampling pode ajudar a melhorar o F1-score.
- **Ajuste de Hiperparâmetros**: Experimentar diferentes valores de hiperparâmetros como número de épocas, taxa de aprendizado e regularização.
- **Modelos mais Complexos**: Explorar modelos mais complexos, como redes neurais mais profundas ou outros algoritmos de machine learning.


#EXTRA

In [28]:
import plotly.graph_objs as go
import plotly.subplots as sp

# Obter o histórico do treinamento
history_dict = history.history

# Criar uma figura com subplots
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=('Acurácia', 'Perda'))

# Gráfico de acurácia
fig.add_trace(go.Scatter(x=list(range(1, len(history_dict['accuracy']) + 1)),
                         y=history_dict['accuracy'],
                         mode='lines+markers',
                         name='Acurácia de Treinamento'), row=1, col=1)

fig.add_trace(go.Scatter(x=list(range(1, len(history_dict['val_accuracy']) + 1)),
                         y=history_dict['val_accuracy'],
                         mode='lines+markers',
                         name='Acurácia de Validação'), row=1, col=1)

# Gráfico de perda
fig.add_trace(go.Scatter(x=list(range(1, len(history_dict['loss']) + 1)),
                         y=history_dict['loss'],
                         mode='lines+markers',
                         name='Perda de Treinamento'), row=1, col=2)

fig.add_trace(go.Scatter(x=list(range(1, len(history_dict['val_loss']) + 1)),
                         y=history_dict['val_loss'],
                         mode='lines+markers',
                         name='Perda de Validação'), row=1, col=2)

# Layout
fig.update_layout(title_text='Progresso do Treinamento ao Longo das Épocas',
                  showlegend=True,
                  xaxis_title='Épocas',
                  yaxis_title='Valor')

# Mostrar o gráfico
fig.show()


##Gráfico de Acurácia (Esquerda)

### Interpretação:
**Acurácia de Treinamento (linha azul)**: Esta linha mostra como a acurácia do seu modelo no conjunto de treinamento variou ao longo das 50 épocas. Podemos ver que a acurácia de treinamento está flutuando ligeiramente em torno de 0.810 (81%), com algumas variações para cima e para baixo.

**Acurácia de Validação (linha vermelha)**: Esta linha representa a acurácia do modelo no conjunto de validação. Assim como a acurácia de treinamento, ela também apresenta algumas flutuações, mas tende a se manter em torno do mesmo valor.

## *Observações*:
As flutuações em ambas as linhas podem ser um sinal de que o modelo está ligeiramente instável e talvez precise de mais ajustes. Em vez de uma curva suave que se aproxima de um valor estável, estamos vendo variações mais significativas entre as épocas, o que pode indicar que o modelo não está convergindo de maneira ideal.

*Estabilidade*: Idealmente, gostaríamos de ver tanto a acurácia de treinamento quanto a de validação convergindo para um valor fixo e subindo gradualmente sem grandes oscilações, o que indicaria que o modelo está aprendendo de forma consistente. Aqui, como as linhas estão um tanto instáveis, pode ser necessário ajustar hiperparâmetros, como a taxa de aprendizado, ou utilizar técnicas como regularização.

## Gráfico de Perda (Direita)
### Interpretação:
**Perda de Treinamento (linha verde):** Esta linha mostra como a perda no conjunto de treinamento evoluiu ao longo das épocas. O gráfico indica que a perda no conjunto de treinamento está muito estável e não apresenta grandes variações, permanecendo em torno de 0.43.

**Perda de Validação (linha roxa)**: Esta linha mostra a perda no conjunto de validação, e, assim como a perda de treinamento, ela é bastante estável, mantendo-se em torno de 0.415.

## *Observações*:
*Estabilidade*: A perda estável no treinamento e na validação pode ser um bom sinal, indicando que o modelo não está sofrendo de overfitting (onde a perda no conjunto de validação aumentaria enquanto a perda no treinamento diminuiria).

*Diferença Entre Perda de Treinamento e Validação*: A diferença entre a perda de treinamento e validação é muito pequena, o que é um bom sinal de que o modelo não está se ajustando demais aos dados de treinamento.